# (2-11) Sequential Recommendation 실습 (하)

<h2 style='font-family:times new roman'>실습 개요</h2>

1) 실습 목적

SASRec과 같은 복잡한 모델을 직접 코딩할 줄 아는 능력도 중요하지만, RecBole과 같은 라이브러리를 효과적으로 활용하는 능력도 또한 연구자와 엔지니어에게 필수적인 역량입니다.
이번에는 RecBole 라이브러리를 사용하여 다양한 순차 추천시스템 모델을 직접 구축할 필요 없이 빠르게 학습시키고 결과를 비교해보는 실습을 진행하겠습니다.

2) 수강 목표
- 목표 1: RecBole 라이브러리의 기본 사용법을 익히고, 이를 통해 다양한 순차 추천 시스템 모델들을 효율적으로 구현하는 방법을 학습합니다.
- 목표 2: RecBole documentation과 같은 문서를 활용하는 방법을 익힙니다.
- 목표 3: 순차 추천 시스템의 다양한 모델들을 실제 데이터셋에 적용해보며, 각 모델의 특성과 성능 차이를 이해합니다.
- 목표 4: 실제 사용 사례에 적합한 모델을 선택하고 최적화하는 방법에 대한 실질적인 경험을 얻을 수 있습니다.

## 실습 목차

* 1. About RecBole
* 2. Data in RecBole
  * 2-1. Atomic File
  * 2-2. Custom Data Loading
* 3. RecBole Modeling
  * 3-1. Config Setting
  * 3-2. Parameter Tuning
* 4. 모델 훈련
  * 4-1. SASRec
  * 4-2. BERT4Rec
  * 4-3. GRU4Rec
* 5. Case Study
* 6. 마치며

###  데이터셋 개요 </b>

* 데이터셋: MovieLens, GoodReads
* 데이터셋 개요: MovieLens 데이터셋은 영화 평가에 기반한 사용자 추천 시스템 연구에 주로 사용되는 데이터 셋입니다. 사용자 리뷰와 함께 다양한 영화에 대한 세부 정보를 포함하고 있어, 추천 알고리즘 개발과 검증에 매우 적합합니다. 이 데이터셋은 사용자별로 다양한 영화에 대한 평점 정보를 제공합니다. GoodReads 데이터셋은 사용자 리뷰와 함께 수많은 책에 대한 상세 정보를 포함하고 있어, 추천 알고리즘의 개발 및 검증에 사용됩니다. 주로 사용자별로 다양한 책에 대한 평점과 리뷰 정보를 제공합니다.
* 데이터셋 저작권: 이 데이터셋은 교육 및 연구 목적으로 무료로 사용할 수 있으며, 상업적 사용과 관련된 구체적인 정보는 MovieLens 공식 웹사이트에서 확인할 수 있습니다. GoodReads 데이터셋은 주로 교육 및 연구 목적으로 사용되며, 이용에 관한 자세한 저작권 정보는 GoodReads의 공식 웹사이트에서 확인할 수 있습니다.



### 환경 설정

- 패키지 설치 및 임포트
```
!pip install recbole
!pip install kmeans_pytorch
!pip install ray
```
```
from recbole.quick_start import run_recbole
from recbole.quick_start.quick_start import load_data_and_model
from recbole.trainer import Trainer
from recbole.utils.case_study import full_sort_scores
from recbole.utils.case_study import full_sort_topk
```

In [ ]:
# 패키지 설치
!pip install recbole
!pip install kmeans_pytorch
!pip install ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 26.1 MB/s eta 0:00:00


In [ ]:
# 라이브러리 임포트
import logging
from logging import getLogger
from recbole.quick_start import run_recbole
from recbole.quick_start.quick_start import load_data_and_model
from recbole.trainer import Trainer
from recbole.utils.case_study import full_sort_scores
from recbole.utils.case_study import full_sort_topk
from recbole.utils import init_seed, init_logger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
import pandas as pd
import gdown
import gzip
import json


## 1. About RecBole
```
💡 목차 개요 : 다양한 추천시스템을 효율적으로 학습 및 평가할 수 있는 RecBole 라이브러리를 소개합니다.
```

#### 📝 설명:

<div align="center">
<img src="https://opengraph.githubassets.com/7e29c60cdad693c99d1b38a9b852352be96549da393605a5e25da693aab3cb45/RUCAIBox/RecBole" width="70%">
<figcaption>RecBole Github</figcaption>
</div>

[RecBole](https://recbole.io/index.html)은 PyTorch를 기반으로 하는 추천시스템 알고리즘 연구 및 개발을 돕기 위해 만들어진 프레임워크입니다. RecBole에서는 일반 추천, 순차 추천, context-aware 추천, 지식 기반 추천 등을 아우르는 다양한 100개 이상의 추천시스템 모델을 제공하여 모델을 직접 구현할 필요 없이 빠르게 모델을 사용하고 학습할 수 있게 도움을 줍니다. 뿐만 아니라, 43개의 벤치마크 데이터셋을 지원하고 fine-tuning과 같은 딥러닝 엔지니어링에서 중요한 피쳐들을 다양하게 제공하고 있어, 추천시스템을 공부하는 수강생 여러분께 유용한 도구가 될 것입니다.

\
<div align="center">
<img src="https://github.com/RUCAIBox/RecBole/raw/master/asset/framework.png" width="60%">
<figcaption>RecBole Overall Architecture</figcaption>
</div>


\
RecBole에서 제공하는 데이터셋 정보는 다음 테이블에서 확인하실 수 있습니다.

| SN | Dataset          | #User       | #Item     | #Interaction | Sparsity | Interaction Type         |
|----|------------------|-------------|-----------|--------------|----------|--------------------------|
| 1  | Adult            | -           | -         | 32,561       | -        | income>=50k [0, 1]       |
| 2  | Alibaba-iFashion | 3,569,112   | 4,463,302 | 191,394,393  | 99.9988% | Click                    |
| 3  | AliEC            | 491,647     | 240,130   | 1,366,056    | 99.9988% | Click                    |
| 4  | Amazon           | -           | -         | -            | -        | Rating [0, 5]            |
| 5  | Amazon_M2        | 3,606,349   | 1,410,675 | 15,306,183   | -        | Click                    |
| 6  | Anime            | 73,515      | 11,200    | 7,813,737    | 99.05%   | Rating [-1, 1-10]        |
| 7  | Avazu            | -           | -         | 40,428,967   | -        | Click                    |
| 8  | BeerAdvocate     | 33,388      | 66,055    | 1,586,614    | 99.9281% | Rating [0, 5]            |
| 9  | Behance          | 63,497      | 178,788   | 1,000,000    | 99.9912% | Likes                    |
| 10 | Book-Crossing    | 105,284     | 340,557   | 1,149,780    | 99.99%   | Rating [0-10]            |
| 11 | Criteo           | -           | -         | 45,850,617   | -        | Click                    |
| 12 | DianPing         | 542,706     | 243,247   | 4,422,473    | 99.9967% | Rating [0, 5]            |
| 13 | DIGINETICA       | 204,789     | 184,047   | 993,483      | 99.99%   | Click                    |
| 14 | Douban           | 738,701     | 28        | 2,125,056    | 89.73%   | Rating [0, 5]            |
| 15 | EndoMondo        | 1,104       | 253,020   | 253,020      | 99.9094% | Workout Logs             |
| 16 | Epinions         | 116,260     | 41,269    | 188,478      | 99.99%   | Rating [1-5]             |
| 17 | Food             | 226,570     | 231,637   | 1,132,367    | 99.9978% | Rating [0, 5]            |
| 18 | Foursquare       | -           | -         | -            | -        | Check-in                 |
| 19 | GoodReads        | 876,145     | 2,360,650 | 228,648,342  | 99.9889% | Rating [0, 5]            |
| 20 | Gowalla          | 107,092     | 1,280,969 | 6,442,892    | 99.99%   | Check-in                 |
| 21 | iPinYou          | 19,731,660  | 163       | 24,637,657   | 99.23%   | View/Click               |
| 22 | Jester           | 73,421      | 101       | 4,136,360    | 44.22%   | Rating [-10, 10]         |
| 23 | KDD2010          | -           | -         | -            | -        | Rating                   |
| 24 | KGRec            | -           | -         | -            | -        | Click                    |
| 25 | Last.FM          | 1,892       | 17,632    | 92,834       | 99.72%   | Click                    |
| 26 | LFM-1b           | 120,322     | 3,123,496 | 1,088,161,692| 99.71%   | Click                    |
| 27 | MIND             | -           | -         | -            | -        | Click                    |
| 28 | ModCloth         | 47,958      | 1,378     | 82,790       | 99.8747% | Rating [0, 5]            |
| 29 | MovieLens        | -           | -         | -            | -        | Rating                   |
| 30 | Music4All-Onion  | 119,140     | 109,269   | 252,984,396  | -        | Click                    |
| 31 | Netflix          | 480,189     | 17,770    | 100,480,507  | 98.82%   | Rating [1-5]             |
| 32 | Phishing websites| -           | -         | 11,055       | -        | Click                    |
| 33 | Pinterest        | 55,187      | 9,911     | 1,445,622    | 99.74%   | -                        |
| 34 | RateBeer         | 29,265      | 110,369   | 2,924,163    | 99.9095% | Overall Rating [0, 20]   |
| 35 | RentTheRunway    | 105,571     | 5,850     | 192,544      | 99.9688% | Rating [0, 10]           |
| 36 | Retailrocket     | 1,407,580   | 247,085   | 2,756,101    | 99.99%   | View/Addtocart/Transaction |
| 37 | Steam            | 2,567,538   | 32,135    | 7,793,069    | 99.99%   | Buy                      |
| 38 | Ta Feng          | 32,266      | 23,812    | 817,741      | 99.89%   | Click                    |
| 39 | Tmall            | 963,923     | 2,353,207 | 44,528,127   | 99.99%   | Click/Buy                |
| 40 | Twitch           | 15,524,309  | 6,161,666 | 474,676,929  | 99.9995% | Click                    |
| 41 | Yahoo Music      | 1,948,882   | 98,211    | 111,557,943  | 99.99%   | Rating [0, 100]          |
| 42 | Yelp             | -           | -         | -            | -        | Rating                   |
| 43 | YOOCHOOSE        | 9,249,729   | 52,739    | 34,154,697   | 99.99%   | Click/Buy                |



\
RecBole에서 제공하는 추천 모델들의 일부는 다음 테이블에서 확인하실 수 있습니다.

| SN | Category                     | Year    | Publication  | Model    | Paper                                                                                                         | Repository   |
|----|------------------------------|---------|--------------|----------|--------------------------------------------------------------------------------------------------------------|--------------|
| 1  | Context-aware Recommendation | 2007    | WWW' 07      | LR       | Predicting Clicks Estimating the Click-Through Rate for New Ads, by Matthew Richardson, Ewa Dominowska, Robert Ragno | RecBole      |
| 2  | Context-aware Recommendation | 2010    | ICDM' 10     | FM       | Factorization Machines, by Steffen Rendle                                                                    | RecBole      |
| 3  | Context-aware Recommendation | 2013    | CIKM' 13     | DSSM     | Learning deep structured semantic models for web search using clickthrough data, by Po-Sen Huang, Xiaodong He, Jianfeng Gao, Li Deng, Alex Acero, Larry Heck | RecBole      |
| 4  | Context-aware Recommendation | 2016    | RecSys' 18   | FFM      | Field-aware Factorization Machines for CTR Prediction, by Yuchin Juan, Yong Zhuang, Wei-Sheng Chin, Chih-Jen Lin | RecBole      |
| 5  | Context-aware Recommendation | 2016    | ECIR' 16     | FNN      | Deep Learning over Multi-field Categorical Data, by Weinan Zhang, Tianming Du, and Jun Wang                   | RecBole      |
| 6  | Context-aware Recommendation | 2016    | ICDM' 16     | PNN      | Product-based neural networks for user response prediction, by Yanru Qu, Han Cai, Kan Ren, Weinan Zhang, Yong Yu, Ying Wen, Jun Wang | RecBole      |
| 7  | Context-aware Recommendation | 2016    | RecSys' 16   | WideDeep | Wide & Deep Learning for Recommender Systems, by Heng-Tze Cheng, Levent Koc, Jeremiah Harmsen, Tal Shaked, Tushar Chandra, Hrishi Aradhye, Glen Anderson, Greg Corrado, Wei Chai, Mustafa Ispir, Rohan Anil, Zakaria Haque, Lichan Hong, Vihan Jain, Xiaobing Liu, Hemal Shah | RecBole      |
| 8  | Context-aware Recommendation | 2017    | IJCAI' 18    | AFM      | Attentional Factorization Machines: Learning the Weight of Feature Interactions via Attention Networks, by Jun Xiao, Hao Ye, Xiangnan He, Hanwang Zhang, Fei Wu, Tat-Seng Chua | RecBole      |
| 9  | Context-aware Recommendation | 2017    | ADKDD' 17    | DCN      | Deep & Cross Network for Ad Click Predictions, by Ruoxi Wang, Bin Fu, Gang Fu, Mingliang Wang                | RecBole      |
| 10 | Context-aware Recommendation | 2017    | IJCAI' 17    | DeepFM   | DeepFM: A Factorization-Machine based Neural Network for CTR Prediction, by Huifeng Guo , Ruiming Tang, Yunming Yey, Zhenguo Li, Xiuqiang He | RecBole      |
| 11 | Context-aware Recommendation | 2017    | SIGIR' 17    | NFM      | Neural Factorization Machines for Sparse Predictive Analytics, by Xiangnan He, Tat-Seng Chua                  | RecBole      |
| 12 | Context-aware Recommendation | 2018    | CIKM' 18     | AutoInt  | AutoInt: Automatic Feature Interaction Learning via Self-Attentive Neural Networks, by Weiping Song, Chence Shi, Zhiping Xiao, Zhijian Duan, Yewen Xu, Ming Zhang, Jian Tang | RecBole      |
| 13 | Context-aware Recommendation | 2018    | SIGKDD' 18   | DIN      | Deep Interest Network for Click-Through Rate Prediction, by Guorui Zhou, Chengru Song, Xiaoqiang Zhu, Ying Fan, Han Zhu, Xiao Ma, Yanghui Yan, Junqi Jin, Han Li, Kun Gai | RecBole      |
| 14 | Context-aware Recommendation | 2018    | WWW' 18      | FwFM     | Field-weighted Factorization Machines for Click-Through Rate Prediction in Display Advertising, by Junwei Pan, Jian Xu, Alfonso Lobos Ruiz, Wenliang Zhao, Shengjun Pan, Yu Sun, Quan Lu | RecBole      |
| 15 | Context-aware Recommendation | 2018    | SIGKDD' 18   | xDeepFM  | xDeepFM: Combining Explicit and Implicit Feature Interactions for Recommender Systems, by Jianxun Lian, Xiaohuan Zhou, Fuzheng Zhang, Zhongxia Chen, Xing Xie, Guangzhong Sun | RecBole      |
| 16 | Context-aware Recommendation | 2019    | AAAI' 19     | DIEN     | Deep Interest Evolution Network for Click-Through Rate Prediction, by Guorui Zhou, Na Mou, Ying Fan, Qi Pi, Weijie Bian, Chang Zhou, Xiaoqiang Zhu, Kun Gai | RecBole      |
| 17 | Context-aware Recommendation | 2019    | CIKM' 19     | FiGNN    | Fi-GNN: Modeling Feature Interactions via Graph Neural Networks for CTR Prediction, by Zekun Li, Zeyu Cui, Shu Wu, Xiaoyu Zhang, Liang Wang | RecBole      |
| 18 | Context-aware Recommendation | 2021    | WWW' 21      | DCNV2    | DCN V2: Improved Deep & Cross Network and Practical Lessons for Web-scale Learning to Rank Systems, by Ruoxi Wang, Rakesh Shivanna, Derek Z. Cheng, Sagar Jain, Dong Lin, Lichan Hong, Ed H. Chi | RecBole      |
| 19 | Context-aware Recommendation | 2023    | WSDM' 23     | KD_DAGFM | Directed Acyclic Graph Factorization Machines for CTR Prediction via Knowledge Distillation, by Zhen Tian, Ting Bai, Zibin Zhang, Zhiyuan Xu, Kangyi Lin, Ji-Rong Wen, Wayne Xin Zhao | RecBole      |
| 20 | Context-aware Recommendation | 2023    | SIGIR'23     | EulerNet | EulerNet: Adaptive Feature Interaction Learning via Euler's Formula for CTR Prediction, by Zhen Tian, Ting Bai, Wayne Xin Zhao, Ji-Rong Wen, Zhao Cao | RecBole      |
| 21 | Cross-domain recommendation | 2008    | SIGKDD'08    | CMF      | Relational Learning via Collective Matrix Factorization, by Ajit P. Singh and Geoffrey J. Gordon | RecBole-CDR  |
| 22 | Cross-domain recommendation | 2013    | PKDD'13      | CLFM     | Cross-Domain Recommendation via Cluster-Level Latent Factor Model, by Sheng Gao, Hao Luo, Da Chen, Shantao Li, Patrick Gallinari and Jun Guo | RecBole-CDR  |
| 23 | Cross-domain recommendation | 2017    | IJCAI'17     | EMCDR    | Cross-Domain Recommendation: An Embedding and Mapping Approach, by Tong Man, Huawei Shen, Xiaolong Jin and Xueqi Cheng | RecBole-CDR  |
| 24 | Cross-domain recommendation | 2018    | CIKM'18      | CoNet    | CoNet: Collaborative Cross Networks for Cross-Domain Recommendation, by Guangneng Hu, Yu Zhang and Qiang Yang | RecBole-CDR  |
| 25 | Cross-domain recommendation | 2018    | IJCAI'18     | DCDCSR   | A Deep Framework for Cross-Domain and Cross-System Recommendations, by Feng Zhu, Yan Wang, Chaochao Chen, Guanfeng Liu, Mehmet Orgun and Jia Wu | RecBole-CDR  |
| 26 | Cross-domain recommendation | 2019    | CIKM'19      | DTCDR    | DTCDR: A Framework for Dual-Target Cross-Domain Recommendation, by Feng Zhu, Chaochao Chen, Yan Wang, Guanfeng Liu and Xiaolin Zheng | RecBole-CDR  |
| 27 | Cross-domain recommendation | 2019    | IJCAI'19     | DeepAPF  | DeepAPF: Deep Attentive Probabilistic Factorization for Multi-site Video Recommendation, by Huan Yan, Xiangning Chen, Chen Gao, Yong Li and Depeng Jin | RecBole-CDR  |
| 28 | Cross-domain recommendation | 2019    | WWW'19       | NATR     | Cross-domain Recommendation Without Sharing User-relevant Data, by Chen Gao, Xiangning Chen, Fuli Feng, Kai Zhao, Xiangnan He, Yong Li and Depeng Jin | RecBole-CDR  |


\
이번 실습에서는 RecBole을 활용하여 대표적인 Sequential Recommendation 모델들을 학습시키고 Case Study를 진행해보겠습니다.



#### 📚 자료:
* [RecBole Sequential Recommendation](https://recbole.io/docs/user_guide/model_intro.html#sequential-recommendation)


Quick Start 예시를 통해 RecBole 라이브러리가 잘 설치되었는지 확인해보고, RecBole이 제공하는 편리함을 간단하게 경험해보는 것으로 실습을 본격적으로 시작해보겠습니다.

코드 한 줄 만으로 일반 추천에 해당하는 [BPR](https://recbole.io/docs/user_guide/model/general/bpr.html) 모델을 RecBole을 설치하면 함께 따라오는 MovieLens 데이터셋의 일종인 ml-100k 데이터셋을 사용하여 training 및 evaluation을 진행할 수 있습니다.



In [ ]:
run_recbole(model='BPR', dataset='ml-100k')

Train    33:  70%|█████████████████▌       | 28/40 [00:00<00:00, 53.61it/s, GPU RAM: 0.03 G/15.77 G]


KeyboardInterrupt: ignored

## 2. Data in RecBole
```
💡 목차 개요 : RecBole을 사용해 Dataset을 준비하고, 모델링하는 방법에 대해 알아봅니다.
```


#### 📝 설명:
RecBole은 MovieLens, BookReads, Douban과 같은 전처리된 다양한 Datasets을 제공하고 있습니다. ([Dataset List](https://recbole.io/dataset_list.html))
직접 준비한 데이터셋을 사용하고 싶은 경우, RecBole의 Atomic File formet에 맞게 가공하여 로딩하면 됩니다. 널리 쓰이는 많은 데이터셋에 적용할 수 있는 converting tool 또한 제공되고 있습니다.

*Atomic Files*란 유연한 방식으로 추천시스템 모델에 대한 입력을 형식화하기 위해 도입된 파일 형식입니다. 구체적으로 .inter, .user., .kg, .link, .net 등 여섯 가지 유형으로 나뉘며, 추천시스템 태스크에 따라 필요로하는 파일 형식이 달라집니다. 이번 실습의 포커스인 순차 추천시스템 모델은 .inter 파일만을 필요로하는데, user_id, item_id, rating, timestamp, review와 같은 column을 가질 수 있는 user-item interaction 데이터셋을 위한 포맷이라고 생각해주시면 됩니다.

### 2-1. Atomic File

> RecBole에서 쓰이는 Atomic File에 대해 알아봅니다.




#### 📚 자료:
* [Atomic Files 소개](https://recbole.io/docs/user_guide/data/atomic_files.html)
* [Running New Dataset](https://recbole.io/docs/v0.1.2/user_guide/usage/running_new_dataset.html)
* [Converting Tool Github](https://github.com/RUCAIBox/RecSysDatasets/tree/master)
* [다운로드 가능한 Atomic Format Dataset](https://drive.google.com/drive/folders/1so0lckI6N6_niVEYaBu-LIcpOdZf99kj)

\

**Pinterest dataset converting 예시 코드**

```
# pinterest dataset 다운로드
!wget https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/pinterest-20.train.rating
!mkdir pinterest
!mv pinterest-20.train.rating ./pinterest/
```
```
# 다운로드 된 dataset Atomic File로 converting 진행
!git clone https://github.com/RUCAIBox/RecDatasets
!pip install -r /content/RecDatasets/conversion_tools/requirements.txt
!python RecDatasets/conversion_tools/run.py --dataset pinterest \
--input_path pinterest --output_path output_data/pinterest \
--convert_inter
```

**Atomic File 예시**
```
# Atomic File 예시
user_id:token	item_id:token
0	2
0	3
0	4
0	5
0	6
0	7
0	8
0	9
0	10
```

### 2-2. Custom Data Loading

> RecBole에서 custom data를 로드하는 법을 알아봅시다.

#### 📝 설명:
RecBole에는 빠른 실험을 위해 여러 데이터셋들이 내장되어 있지만, 새로운 커스텀 데이터셋을 사용해야 할 필요도 있습니다. 이를 위해서는 Raw 데이터셋을 알맞은 Atomic File 형태로 직접 변환해야 합니다. 간단하게 `*.inter` 파일만 생성해서 테스트해보겠습니다. 이외에도 `*.user`, `*.item` 과 같이 content 정보도 담을 수 있습니다.

새 데이터셋을 사용하려면 config에서 데이터 경로를 설정해야 합니다. Atomic File의 이름, Atomic File이 포함된 directory 이름 및 _ `config['dataset']`_은 동일해야 하며, config의 `data_path`는 Atomic File이 포함된 디렉터리의 상위 directory여야 합니다.

예를 들어, 다음 Atomic File에 대응하는
```
~/xxx/yyy/ml-1m/
├── ml-1m.inter
├── ml-1m.item
├── ml-1m.kg
├── ml-1m.link
└── ml-1m.user
```
에 대응하는 config 옵션은 다음과 같습니다.

```
data_path: ~/xxx/yyy/
dataset: ml-1m
```

#### 📚 자료:
* [RecBole Documentation](https://recbole.io/docs/user_guide/usage/running_new_dataset.htmlf)



In [ ]:
# 지난 Sequential Recommendation 실습에서 사용했던 데이터 사용
file_id = "1JO1Y3McBAPQuXHG1tezbk1n7_MnegA_1"
output = "./goodreads_reviews_spoiler.json.gz" # 저장 위치 및 저장할 파일 이름
gdown.download(id=file_id, output=output, quiet=False)

def load_data(file_name):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)
    return data

review_data = load_data(output) # 약 1분 가량 소요됨

Downloading...
From: https://drive.google.com/uc?id=1JO1Y3McBAPQuXHG1tezbk1n7_MnegA_1
To: /content/goodreads_reviews_spoiler.json.gz
100%|██████████| 620M/620M [00:08<00:00, 71.9MB/s]


In [ ]:
df = pd.DataFrame(review_data)
df.head()

,user_id,timestamp,review_sentences,rating,has_spoiler,book_id,review_id
0,8842281e1d1347389f2ab93d60773d4d,2017-08-30,"[[0, This is a special book.], [0, It started ...",5,True,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb
1,8842281e1d1347389f2ab93d60773d4d,2017-03-22,"[[0, Recommended by Don Katz.], [0, Avail for ...",3,False,16981,a5d2c3628987712d0e05c4f90798eb67
2,8842281e1d1347389f2ab93d60773d4d,2017-03-20,"[[0, A fun, fast paced science fiction thrille...",3,True,28684704,2ede853b14dc4583f96cf5d120af636f
3,8842281e1d1347389f2ab93d60773d4d,2016-11-09,"[[0, Recommended reading to understand what is...",0,False,27161156,ced5675e55cd9d38a524743f5c40996e
4,8842281e1d1347389f2ab93d60773d4d,2016-04-25,"[[0, I really enjoyed this book, and there is ...",4,True,25884323,332732725863131279a8e345b63ac33e


In [ ]:
# user_id, book_id, timestamp만 남기고 간단히 전처리
# 커스텀 데이터 생성 테스트 용도 이므로 10%만 샘플링해서 처리함
df = df.sample(frac=0.1, random_state=42)[["user_id", "book_id", "timestamp"]].reset_index(drop=True)

In [ ]:
unique_users = df["user_id"].unique()
unique_items = df["book_id"].unique()

user_id2idx = {v: k for k, v in enumerate(unique_users)}
item_id2idx = {v: k for k, v in enumerate(unique_items)}

df["user_idx"] = df["user_id"].map(user_id2idx)
df["item_idx"] = df["book_id"].map(item_id2idx)

In [ ]:
df.head()

,user_id,book_id,timestamp,user_idx,item_idx
0,818a07d4b1a085d65a3851c9f68f148d,28587986,2017-02-28,0,0
1,eac49beafd4485d9c564dc8fab576fb8,15844362,2014-10-29,1,1
2,c6f39599f1c5d67d491a86fa6bafb816,17571742,2013-08-31,2,2
3,900c1edf2ede90f385872938ce6f16c9,22370569,2014-07-04,3,3
4,8dab3f118616eb3f550a927f35533905,14460,2015-02-09,4,4


In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp']).astype(int) / 10**9 # covert to unix timestamp (sec)

In [ ]:
df = df.sort_values(["user_idx", "item_idx", "timestamp"])
df = df.rename(columns={"user_idx": "user_idx:token", "item_idx": "item_idx:token", "timestamp":"timestamp:float"})

In [ ]:
df

,user_id,book_id,timestamp:float,user_idx:token,item_idx:token
0,818a07d4b1a085d65a3851c9f68f148d,28587986,1.488240e+09,0,0
580,818a07d4b1a085d65a3851c9f68f148d,186074,1.486426e+09,0,39
91690,818a07d4b1a085d65a3851c9f68f148d,17131869,1.468973e+09,0,41
133088,818a07d4b1a085d65a3851c9f68f148d,22544764,1.462925e+09,0,119
87986,818a07d4b1a085d65a3851c9f68f148d,8235178,1.326326e+09,0,163
...,...,...,...,...,...
137455,3321536067599afc17262454fed3b941,29579,1.422403e+09,15884,8144
137534,fba00d66cd19863d39e484490bee78f7,4981,1.303517e+09,15885,1032
137667,44832870ae9ea9e4e73c317007faae03,19504931,1.438733e+09,15886,278
137742,fbd77069ce0061b31624158bb36e80c4,9601799,1.397606e+09,15887,21350


In [ ]:
!mkdir ./recbole_data
df[["user_idx:token", "item_idx:token", "timestamp:float"]].to_csv("./recbole_data/recbole_data.inter", sep='\t', index=False)

In [ ]:
pd.read_csv("./recbole_data/recbole_data.inter", sep="\t")

,user_idx:token,item_idx:token,timestamp:float
0,0,0,1.488240e+09
1,0,39,1.486426e+09
2,0,41,1.468973e+09
3,0,119,1.462925e+09
4,0,163,1.326326e+09
...,...,...,...
137798,15884,8144,1.422403e+09
137799,15885,1032,1.303517e+09
137800,15886,278,1.438733e+09
137801,15887,21350,1.397606e+09


In [ ]:
parameter_dict = {
    'data_path': '/content/',
    'USER_ID_FIELD': 'user_idx',
    'ITEM_ID_FIELD': 'item_idx',
    'TIME_FIELD': 'timestamp',
    'user_inter_num_interval': "[5,Inf)",
    'item_inter_num_interval': "[5,Inf)",
    'load_col': {'inter': ['user_idx', 'item_idx', 'timestamp']},
    'train_neg_sample_args': None,
    'epochs': 5,
    'stopping_step':3,

    'eval_batch_size': 1024,
    'MAX_ITEM_LIST_LENGTH': 50,

    # EVALUATION SETTINGS
    'eval_args': {
        'split': {'RS': [9, 1, 0]}, # ratio-based sampling 9:1:0
        'group_by': 'user', # 사용자 기준
        'order': 'TO', # time-ordering
        'mode': 'full' # ['full','unixxx','popxxx'] 등이 있음
        # e.g., 'uni100': uniformly sample 100 negative items for each positive item in testing set
     },
    'device': 'cuda'
}

config = Config(model='GRU4Rec', dataset='recbole_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()

# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

logger.info(config)

In [ ]:
dataset = create_dataset(config)
logger.info(dataset)

In [ ]:
train_data, valid_data, test_data = data_preparation(config, dataset)

In [ ]:
# model loading and initialization
model = GRU4Rec(config, train_data.dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
#best_valid_score, best_valid_result = trainer.fit(train_data)

In [ ]:
best_valid_score, best_valid_result

(0.0047,
 OrderedDict([('recall@10', 0.0147),
              ('mrr@10', 0.0047),
              ('ndcg@10', 0.007),
              ('hit@10', 0.0147),
              ('precision@10', 0.0015)]))

In [ ]:
import gc
gc.collect()

26

## 3. RecBole Modeling

```
💡 목차 개요 : RecBole을 사용하여 모델을 훈련하기 위해서 데이터셋 준비 외, 각 모델에 대한 RecBole documentation을 참고하여 Config File을 작성하는 방법과 Hyper Parameter에 대한 정보를 알아야 합니다.
```

* 2-1. Config Setting
* 2-2. Parameter Tuning

### 3-1. Config Setting
> RecBole을 통한 Config Setting을 예시 코드를 통해 알아보겠습니다.

#### 📚 자료:
* [Config File in RecBole](https://recbole.io/docs/user_guide/config_settings.html?highlight=config)

#### 📝 설명:

Config setting 과정은 크게 [Environment settings](https://recbole.io/docs/user_guide/config/environment_settings.html), [Data settings](https://recbole.io/docs/user_guide/config/data_settings.html), [Training settings](https://recbole.io/docs/user_guide/config/training_settings.html), [Evaluation settings](https://recbole.io/docs/user_guide/config/evaluation_settings.html), 그리고 [Parameter Configuration](https://recbole.io/docs/user_guide/config/parameters_configuration.html)으로 구성되어 있습니다. Config를 세팅하는 방법은 config file, parameter dicts, command line으로 총 세 가지가 있습니다. 자세한 사용법은 [링크](https://recbole.io/docs/user_guide/config/parameters_configuration.html)를 확인하시면 됩니다.
모델마다 세팅해야 하는 config가 조금씩 차이가 있습니다. 사용하려는 모델 config에 대한 정보는 [RecBole 공식 다큐멘테이션](https://recbole.io/docs/index.html) 내에서 검색하여 확인해볼 수 있습니다.



**Config 예시**
```
# dataset config : General Recommendation
USER_ID_FIELD: user_id
ITEM_ID_FIELD: item_id
load_col:
    inter: [user_id, item_id]

# model config
embedding_size: 64

# Training and evaluation config
epochs: 2
train_batch_size: 4096
eval_batch_size: 4096
train_neg_sample_args: None
eval_args:
    group_by: user
    order: RO
    split: {'RS': [0.8,0.1,0.1]}
    mode: full
metrics: ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk: 10
valid_metric: MRR@10
metric_decimal_place: 4

# Filtering
user_inter_num_interval: "[1, 50]"
item_inter_num_interval: "[1, 50]"
filter_inter_by_user_or_item: True
```



### 3-2. Parameter Tuning
> 예시 코드를 통해 RecBole 라이브러리를 사용한 Parameter Tuning에 대해 간단히 알아보겠습니다.
#### 📝 설명:
RecBole에서 제공하는 하이퍼파라미터 튜닝 기능으로 모델을 최적화할 수 있습니다. 이를 위해 라이브리 내부에서 Hyperopt와 Ray 라이브러리를 사용하며, 그리드 탐색, 랜덤 탐색, 베이지안 하이퍼옵트 등 세 가지 튜닝 방식 외에도 사용자 정의 탐색 방법 (user-defined)을 제공합니다. 자세한 내용은 [링크](https://recbole.io/docs/user_guide/usage/parameter_tuning.html?highlight=hyperparameter)를 참고하시면 됩니다.



**Hyperopt 튜닝 예시**
```
from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function

hp = HyperTuning(objective_function=objective_function, algo='exhaustive', early_stop=10,
                max_evals=100, params_file='model.hyper', fixed_config_file_list=['example.yaml'])

# run
hp.run()
# export result to the file
hp.export_result(output_file='hyper_example.result')
# print best parameters
print('best params: ', hp.best_params)
# print best result
print('best result: ')
print(hp.params2result[hp.params2str(hp.best_params)])
```
```
python run_hyper.py --config_files=[config_files] --params_file=[params_file] --output_file=[output_file] --tool=Hyperopt
```
\

**Ray 튜닝 예시**
```
from ray import tune

result = tune.run(
    tune.with_parameters(objective_function, config_file_list=config_file_list),
    config=config,
    num_samples=5,
    log_to_file=args.output_file,
    scheduler=scheduler,
    local_dir=local_dir,
    resources_per_trial={
        "gpu": 1
    }
)
best_trial = result.get_best_trial("recall@10", "max", "last")
print("best params: ",best_trial.config)
print("best result: ",best_trial.last_result)
```
```
python run_hyper.py --config_files=[config_files] --output_file=[output_file] --tool=Ray
```

### 4. 모델 훈련
```
💡 목차 개요 :MovieLens 데이터셋을 이용한 모델 훈련을 진행하겠습니다.
```

* 4-1. SASRec
* 4-2. BERT4Rec
* 4-4. GRU4Rec


#### 📚 자료:
훈련시킬 모델들은 다음과 같습니다. 도큐먼트에서 모델별로 세팅해야 하는 파라미터를 확인하실 수 있습니다.
* [SASRec](https://recbole.io/docs/user_guide/model/sequential/sasrec.html?highlight=sasrec)
* [BERT4Rec](https://recbole.io/docs/user_guide/model/sequential/bert4rec.html?highlight=bert4rec)
* [GRU4Rec](https://recbole.io/docs/user_guide/model/sequential/gru4rec.html?highlight=gru4rec)

In [ ]:
# parameter setting
parameter_dict = {
    # dataset config : Sequential Recommendation
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'load_col': {
        'inter': ['user_id', 'item_id', 'timestamp']
    },
    'LIST_SUFFIX': '_list',
    'ITEM_LIST_LENGTH_FIELD': 'item_length',
    'MAX_ITEM_LIST_LENGTH': 50,

    # model config
    'embedding_size': 64,
    'hidden_size': 128,
    'num_layers': 1,
    'dropout_prob': 0.3,
    'loss_type': 'CE',

    # Training and evaluation config
    'epochs': 5,
    'train_batch_size': 4096,
    'eval_batch_size': 4096,
    'train_neg_sample_args': None,
    'eval_args': {
        'group_by': 'user',
        'order': 'TO',
        'split': {
            'LS': 'valid_and_test'
        },
        'mode': 'full'
    },
    'metrics': ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision'],
    'topk': 10,
    'valid_metric': 'MRR@10'
}

### 4-1. SASRec
> SASRec은 Transformer 아키텍처를 기반으로 하며, 사용자의 이전 상호작용을 분석하여 개인화된 추천을 생성합니다.

<div align="center">
<img src="https://recbole.io/docs/_images/sasrec.png" width="50%">
<figcaption>SASRec Architecture</figcaption>
</div>

In [ ]:
# SASRec 모델 훈련
run_recbole(model='SASRec', dataset='ml-100k', config_dict=parameter_dict)

command line args [-f /root/.local/share/jupyter/runtime/kernel-727ac881-2338-49f2-823f-99bca4fbcd5e.json] will not be used in RecBole
Evaluate   : 100%|███████████████████████████| 1/1 [00:00<00:00, 48.38it/s, GPU RAM: 5.36 G/15.77 G]


{'best_valid_score': 0.0349,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@10', 0.122),
              ('mrr@10', 0.0349),
              ('ndcg@10', 0.0548),
              ('hit@10', 0.122),
              ('precision@10', 0.0122)]),
 'test_result': OrderedDict([('recall@10', 0.0965),
              ('mrr@10', 0.0279),
              ('ndcg@10', 0.0436),
              ('hit@10', 0.0965),
              ('precision@10', 0.0097)])}

### 4-2. BERT4Rec
>BERT4Rec은 BERT 모델을 기반으로 하여, 양방향 컨텍스트를 고려하여 사용자의 행동 시퀀스를 모델링합니다.

<div align="center">
<img src="https://recbole.io/docs/_images/bert4rec.png" width="60%">
<figcaption>BERT4Rec Architecture</figcaption>
</div>

In [ ]:
# BERT4Rec 모델 훈련
run_recbole(model='BERT4Rec', dataset='ml-100k', config_dict=parameter_dict)

command line args [-f /root/.local/share/jupyter/runtime/kernel-727ac881-2338-49f2-823f-99bca4fbcd5e.json] will not be used in RecBole
Evaluate   : 100%|███████████████████████████| 1/1 [00:00<00:00, 11.63it/s, GPU RAM: 5.36 G/15.77 G]


{'best_valid_score': 0.0322,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@10', 0.106),
              ('mrr@10', 0.0322),
              ('ndcg@10', 0.049),
              ('hit@10', 0.106),
              ('precision@10', 0.0106)]),
 'test_result': OrderedDict([('recall@10', 0.0795),
              ('mrr@10', 0.0234),
              ('ndcg@10', 0.036),
              ('hit@10', 0.0795),
              ('precision@10', 0.008)])}

### 4-3. GRU4Rec
>GRU4Rec은 GRU (Gated Recurrent Unit)를 활용하여 사용자의 행동 시퀀스를 모델링합니다.

<div align="center">
<img src="https://recbole.io/docs/_images/gru4rec.png" width="50%">
<figcaption>GRU4Rec Architecture</figcaption>
</div>

In [ ]:
# GRU4Rec 모델 훈련
run_recbole(model='GRU4Rec', dataset='ml-100k', config_dict=parameter_dict)

command line args [-f /root/.local/share/jupyter/runtime/kernel-727ac881-2338-49f2-823f-99bca4fbcd5e.json] will not be used in RecBole
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:1102: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  result = _VF.gru(input, hx, self._flat_weights, self.bias, self.num_layers,
Evaluate   : 100%|███████████████████████████| 1/1 [00:00<00:00, 44.82it/s, GPU RAM: 6.94 G/15.77 G]


{'best_valid_score': 0.011,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('recall@10', 0.0361),
              ('mrr@10', 0.011),
              ('ndcg@10', 0.0167),
              ('hit@10', 0.0361),
              ('precision@10', 0.0036)]),
 'test_result': OrderedDict([('recall@10', 0.0403),
              ('mrr@10', 0.0123),
              ('ndcg@10', 0.0188),
              ('hit@10', 0.0403),
              ('precision@10', 0.004)])}

참고: 이번 실습에서처럼 모든 config를 반드시 입력해야만 학습이 되는 것이 아니라, 기본적인 세팅만 해주어도 학습이 가능합니다. (설정하지 않은 파라미터는 dafault 값으로 설정됨) 필수적으로 입력해야 하는 config는 각 모델의 RecBole 도큐먼트 페이지에 명시되어 있습니다.

## 5. Case Study
```
💡 목차 개요 : Case Study를 통해 결과 예제를 확인해봅니다.
```

In [ ]:
!ls

goodreads_reviews_spoiler.json.gz  log	log_tensorboard  recbole_data  sample_data  saved


In [ ]:
!ls ./saved

BERT4Rec-Jan-03-2024_10-37-03.pth  GRU4Rec-Jan-03-2024_10-37-43.pth
BPR-Jan-03-2024_10-17-49.pth	   SASRec-Jan-03-2024_10-35-53.pth
GRU4Rec-Jan-03-2024_10-28-14.pth


In [ ]:
model_path = '/content/saved/SASRec-Jan-03-2024_10-35-53.pth' # 여기서 모델 파일 경로는 예시입니다.
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(model_file=model_path)
trainer = Trainer(config, model)

In [ ]:
## case study를 위한 임의의 user id를 지정합니다.
# external_ids의 값은 string이어야 합니다.
# 해당 id는 dataset.token2id를 통해 external_id -> internal_id 로 변환됩니다.
# 여기에서 external_id가 우리가 원래 봤던 아이디를 의미하고,
# internal_id가 RecBole 내부에서 맵핑된 id 체계를 의미합니다.
external_ids = ['111', '21', '39']
uid_series = dataset.token2id(dataset.uid_field, external_ids)
print ("external user id: {}, internal id : {}".format(external_ids, uid_series))

external user id: ['111', '21', '39'], internal id : [190 176 327]


참고: token2id()를 통해 외부 user id를 내부 user id로 변환해주어야 합니다.

In [ ]:
# 모든 아이템에 대한 score 확인
score = full_sort_scores(uid_series, model, test_data, device=config['device'])
print(score)
# item ['242', '302'] 에 대한 '111' 사용자의 결과 확인하기
print(score[0, dataset.token2id(dataset.iid_field, ['242', '302'])])

tensor([[   -inf,  4.2045,  5.5934,  ..., -1.5458, -1.2213, -1.4846],
        [   -inf, -1.1400, -0.9971,  ...,  0.0424,  0.1985,  0.3295],
        [   -inf,  3.7323,  4.3726,  ..., -1.4974, -1.3569, -1.6947]],
       device='cuda:0')
tensor([4.2045, 5.5934], device='cuda:0')


참고: [pad]와 이전 아이템들 (중복을 피하기 위한)의 점수는 -inf로 설정됩니다.

In [ ]:
topk_score, topk_iid_list = full_sort_topk(uid_series, model, test_data, k=10, device=config['device'])
print("scores of top 10 items for each user:\n ", topk_score)
# 선호도에 따라, 각 user 에게 추천되는 아이템이 다른 것을 관찰할 수 있습니다.
print("\ninternal ids:\n ", topk_iid_list)
external_item_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu())
print("\nexternal ids:\n ", external_item_list)

scores of top 10 items for each user:
  tensor([[5.9979, 5.7241, 5.6847, 5.6710, 5.6246, 5.5934, 5.5836, 5.5322, 5.5096,
         5.4486],
        [3.9142, 3.7601, 3.6954, 3.6864, 3.6383, 3.6264, 3.6235, 3.6033, 3.5812,
         3.5752],
        [6.3208, 6.2158, 6.0289, 5.9517, 5.8560, 5.8133, 5.8114, 5.7333, 5.6667,
         5.5781]], device='cuda:0')

internal ids:
  tensor([[ 653,   96,  290,  158,   61,    2,   99,  140,  281,  492],
        [ 364,   15,  608,  835,  198,  979,  605,  206, 1020,  283],
        [ 176,   96,  361,   99,   52,  653,  145,  118,  275,  861]],
       device='cuda:0')

external ids:
  [['300' '294' '286' '258' '288' '302' '328' '268' '333' '269']
 ['94' '29' '386' '67' '554' '391' '569' '401' '722' '577']
 ['748' '294' '289' '328' '322' '300' '323' '245' '678' '326']]


## 6. 마치며
이번 실습에서는 RecBole을 활용하여 데이터 전처리, 모델 선택 및 튜닝, 그리고 결과 해석에 이르는 추천시스템의 전체 파이프라인을 쉽고 빠르게 경험해 보았습니다. 또한, 학습된 모델의 Case Study를 통해 각 사용자에게 최종적으로 추천되는 아이템들이 어떻게 개인 선호도에 따라 상이한 결과를 보여주는지 관찰했습니다. 이번 실습을 통해 수강생 여러분께서 추천시스템에 대해 보다 깊은 이해를 얻어가시길 바라겠습니다.

## References

- [RecBole 공식 홈페이지](https://recbole.io/)
- [RecBole Github repository](https://github.com/RUCAIBox/RecBole)

## Required Package

- recbole==1.2.0
- kmeans-pytorch==0.3
- ray==2.9.0

## 콘텐츠 라이선스

저작권 : <font color='blue'> <b> ©2023 by Upstage X fastcampus Co., Ltd. All rights reserved.</font></b>

<font color='red'><b>WARNING</font> : 본 교육 콘텐츠의 지식재산권은 업스테이지 및 패스트캠퍼스에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다. </b>